### Deploy Web App on Azure Container Services (AKS)
In this notebook we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the ACS cluster. Then we will check everything is working by sending an image to it and getting it scored.

The process is split into the following steps:
* [Define our resource names](#section1)
* [Login to Azure](#section2)
* [Create the ACS](#section3)
* [Create a tunnel to the head node](#section4)
* [Create a JSON schema of our APP and push it to the cluster](#section5)
* [Test our app](TestWebApp.ipynb)
* [Tear it all down](#section7)

This guide assumes is designed to be run on linux and requires that the Azure CLI is installed.

<a id='section1'></a>

## Setup
Below are the various name definitions for the resources needed to setup ACS as well as the name of the Docker image we will be using.

**Some outputs (and inputs) below have been hidden/masked for confidentiality**

In [1]:
resource_group = "msaksrg" # Feel free to modify these
aks_name = "msAKSTFCluster"
location = "eastus"

image_name = 'masalvar/tfresnet-gpu' 
selected_subscription = "'Team Danielle Internal'" # If you have multiple subscriptions select 
                                                   # the subscription you want to use here

<a id='section2'></a>

## Azure account login
The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [2]:
!az login -o table

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FNGYXPRU3 to authenticate.
CloudName    Name                           State    TenantId                              IsDefault
-----------  -----------------------------  -------  ------------------------------------  -----------
AzureCloud   Boston DS Dev                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Azure Internal - London        Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Team Danielle Internal         Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47  True
AzureCloud   Visual Studio Enterprise       Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Boston Engineering             Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   ADLTrainingMS                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   PhillyExt                      Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   Ads Eng Big Data Sub

In [3]:
!az account set --subscription $selected_subscription

In [4]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "masalvar@microsoft.com",
    "type": "user"
  }
}


In [5]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


<a id='section3'></a>

## Create resources and dependencies

### Create resource group
Azure encourages the use of groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the group.

In [6]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/msaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "msaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


In [29]:
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

{- Finished ..
  "additionalProperties": {},
  "agentPoolProfiles": [
    {
      "additionalProperties": {},
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "msAKSTFClu-msaksrg-edf507",
  "fqdn": "msakstfclu-msaksrg-edf507-26f4c0b4.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/msaksrg/providers/Microsoft.ContainerService/managedClusters/msAKSTFCluster",
  "kubernetesVersion": "1.7.9",
  "linuxProfile": {
    "additionalProperties": {},
    "adminUsername": "azureuser",
    "ssh": {
      "additionalProperties": {},
      "publicKeys": [
        {
          "additionalProperties": {},
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDVfKBWPBKS84wluD3DJ0t3hepO2F13pz1VI5d4c7Tn4d80rSKJkF2L

Install cli from comamnd prompt

In [9]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


In [60]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "masalvar/tfresnet-gpu",
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "name": "bin",
                              "mountPath":"/usr/local/nvidia/bin" 
                          },
                          {
                              "name": "lib",
                              "mountPath":"/usr/local/nvidia/lib64" 
                          },
                          {
                              "name": "libcuda",
                              "mountPath":"/usr/lib/x86_64-linux-gnu/libcuda.so.1" 
                          },
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "bin",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384/bin"
                      },
                  },
                  {
                      "name": "lib",
                      "hostPath":{
                          "path":"/usr/lib/nvidia-384"
                      },
                  },
                  {
                      "name": "libcuda",
                      "hostPath":{
                          "path":"/usr/lib/x86_64-linux-gnu/libcuda.so.1"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [61]:
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4,sort_keys=True)
        outfile.write('\n\n')

In [62]:
write_json_to_file(app_template, 'az-dl.json')

In [63]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

In [64]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                            }
                        ],
                        "image": "masalvar/tfresnet-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
            

In [35]:
!az aks get-credentials --resource-group=$resource_group --name=$aks_name

Merged "msAKSTFCluster" as current context in /home/mat/.kube/config


In [65]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-27496346-0   Ready     agent     44m       v1.7.9


In [66]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   heapster-2574232661-1dx42               2/2       Running   0          41m
kube-system   kube-dns-v20-2253765213-3kb0s           3/3       Running   0          42m
kube-system   kube-dns-v20-2253765213-p80ng           3/3       Running   0          42m
kube-system   kube-proxy-9zd4s                        1/1       Running   0          42m
kube-system   kube-svc-redirect-c8klv                 1/1       Running   0          42m
kube-system   kubernetes-dashboard-2898242510-9l409   1/1       Running   0          42m
kube-system   tunnelfront-180102643-hn69h             1/1       Running   0          42m


In [67]:
!kubectl create -f az-dl.json

deployment "azure-dl" created
service "azure-dl" created


In [69]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-3880299103-jsn4n               1/1       Running   0          11m
kube-system   heapster-2574232661-1dx42               2/2       Running   0          53m
kube-system   kube-dns-v20-2253765213-3kb0s           3/3       Running   0          54m
kube-system   kube-dns-v20-2253765213-p80ng           3/3       Running   0          54m
kube-system   kube-proxy-9zd4s                        1/1       Running   0          54m
kube-system   kube-svc-redirect-c8klv                 1/1       Running   0          54m
kube-system   kubernetes-dashboard-2898242510-9l409   1/1       Running   0          54m
kube-system   tunnelfront-180102643-hn69h             1/1       Running   0          54m


In [70]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                             SOURCE                                 MESSAGE
56m         1h           7         aks-nodepool1-27496346-0.151e4321d9812de1    Node                                     Normal    NodeHasSufficientDisk              kubelet, aks-nodepool1-27496346-0      Node aks-nodepool1-27496346-0 status is now: NodeHasSufficientDisk
56m         1h           7         aks-nodepool1-27496346-0.151e4321d9818fef    Node                                     Normal    NodeHasSufficientMemory            kubelet, aks-nodepool1-27496346-0      Node aks-nodepool1-27496346-0 status is now: NodeHasSufficientMemory
56m         1h           7         aks-nodepool1-27496346-0.151e4321d981b123    Node                                     Normal    NodeHasNoDiskPressure              kubelet, aks-nodepool1-27496346-0      Node aks-nodepool1-27496346-0 status

In [55]:
!kubectl logs azure-dl-2914933029-tlnmp

2018-03-22 14:41:03,137 CRIT Supervisor running as root (no user in config file)
2018-03-22 14:41:03,139 INFO supervisord started with pid 7
2018-03-22 14:41:04,141 INFO spawned: 'program_exit' with pid 17
2018-03-22 14:41:04,143 INFO spawned: 'nginx' with pid 18
2018-03-22 14:41:04,144 INFO spawned: 'gunicorn' with pid 19
2018-03-22 14:41:05,174 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-03-22 14:41:09,192 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
Selected GPU[0] Tesla K80 as the process wide default device.
Initialising
Model loading time: 13501.91 ms
{"timestamp": "2018-03-22T14:41:18.246347Z", "message": "Model loading time: 13501.91 ms", "host": "azure-dl-2914933029-tlnmp", "path": "/code/driver.py", "tags": [], "level": "INFO", "logger": "cntk_svc_logger", "stack_info": null}
{"timestamp": "2018-03-22T14:41:18.250653Z", "message": " * Running on http://127.0.

In [71]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP    EXTERNAL-IP    PORT(S)        AGE
azure-dl   LoadBalancer   10.0.155.14   13.82.238.75   80:30532/TCP   11m


<a id='section7'></a>

## Tear it all down 
Once you are done with your cluster you can use the following two commands to destroy it all.

In [72]:
!kubectl delete -f az-dl.json

deployment "azure-dl" deleted
service "azure-dl" deleted


In [75]:
!az aks delete -n $aks_name -g $resource_group -y

inished ..

In [76]:
!az group delete --name $resource_group -y

inished ..